<a href="https://colab.research.google.com/github/AlinaTkachenko/knowledge_graph_embedding/blob/main/MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 21.1 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y mysql-server

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [891 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [3]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
#!mysql_secure_installation

In [8]:
#!mysql -e "CREATE DATABASE test_db;"
#!mysql -e "CREATE USER 'test_user'@'localhost' IDENTIFIED BY 'test_pass';"
!mysql -e "GRANT ALL PRIVILEGES ON test_db.* TO 'test_user'@'localhost';"
!mysql -e "FLUSH PRIVILEGES;"


ERROR 1396 (HY000) at line 1: Operation CREATE USER failed for 'test_user'@'localhost'


In [9]:
# Allow local infile and set secure_file_priv in MySQL configuration
!sed -i '/\[mysqld\]/a local_infile=1\nsecure_file_priv=""' /etc/mysql/mysql.conf.d/mysqld.cnf

In [23]:
!service mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


Parse JSON ito CSV

In [14]:
import pandas as pd
import json
import re

# Function to parse the keywords column with error handling
def parse_keywords(row):
    movie_id = row['id']
    try:
        # Replace single quotes with double quotes and remove any malformed parts
        keywords_str = re.sub(r"(?<!\\)'", "\"", row['keywords'])
        keywords_list = json.loads(keywords_str)
        parsed_keywords = []
        for keyword in keywords_list:
            parsed_keywords.append({'movie_id': movie_id, 'keyword_id': keyword['id'], 'keyword': keyword['name']})
        return parsed_keywords
    except json.JSONDecodeError as e:
        return []

# Load the CSV file
file_path = '/content/keywords.csv'  # Update this with the actual path to your CSV file
keywords_df = pd.read_csv(file_path, encoding='utf-8')

# Print the first few rows of the DataFrame
print(keywords_df.head())

# Apply the function to each row and collect the results
parsed_data = []
for index, row in keywords_df.iterrows():
    parsed_data.extend(parse_keywords(row))

# Create a new DataFrame with the parsed data
parsed_keywords_df = pd.DataFrame(parsed_data)

# Save the new DataFrame to a CSV file
parsed_keywords_df.to_csv('parsed_keywords.csv', index=False)

print(parsed_keywords_df.head())


      id                                           keywords
0    862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1   8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2  15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3  31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4  11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
   movie_id  keyword_id     keyword
0       862         931    jealousy
1       862        4290         toy
2       862        5202         boy
3       862        6054  friendship
4       862        9713     friends


# JSON into valid JSON

In [ ]:
import pandas as pd
import json
import re

df = pd.read_csv('/content/sample_data/keywords.csv')

# Function to validate and correct JSON strings
def validate_and_correct_json(data):
    if pd.isna(data):
        return '[]'
    # Replace single quotes with double quotes
    data = data.replace("'", '"')
    # Escape any embedded double quotes within strings
    data = re.sub(r'(?<!\\)"', r'\\"', data)
    # Correctly escape backslashes that are not followed by a valid escape character
    data = re.sub(r'\\(?!["\\/bfnrtu])', r'\\\\', data)
    # Handle specific invalid escape sequences
    data = data.replace('\\xa0', ' ')
    # Use a try-except block to catch invalid JSONs and attempt correction
    try:
        json.loads(data)
    except ValueError as e:
        # Additional handling for common issues
        data = data.replace('\\"', '"')  # Unescape quotes for JSON parsing
        data = re.sub(r'\\u([0-9A-Fa-f]{4})', lambda m: chr(int(m.group(1), 16)), data)  # Handle Unicode escapes
        try:
            json.loads(data)
        except ValueError as e:
            print(f"Still Invalid JSON: {data}\nError: {e}")
            return '[]'  # Return an empty JSON array if parsing fails
    return data

# Apply the function to the 'keywords' column
df['keywords'] = df['keywords'].apply(validate_and_correct_json)



# Save the preprocessed DataFrame back to CSV
preprocessed_filename = '/content/sample_data/preprocesed_keywords.csv'
df.to_csv(preprocessed_filename, index=False)


In [24]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="test_user",
    password="test_pass",
    database="test_db",
    allow_local_infile=True  # Enable local infile
)
cursor = conn.cursor()

In [25]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS keywords (
    id INT,
    keyword_id INT,
    keyword VARCHAR(255)
)
""")

conn.commit()

CSV file contains JSON data that uses single quotes (') instead of double quotes ("). MySQL expects JSON data to use double quotes for property names and string values.

```
# This is formatted as code
```



In [26]:
#Truncate delete all data from table
cursor.execute("""
TRUNCATE TABLE keywords;
""")
conn.commit()

cursor.execute("""
LOAD DATA LOCAL INFILE '/content/parsed_keywords.csv'
INTO TABLE keywords
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES
(id, keyword_id, keyword)
""")

conn.commit()


In [27]:
# Select and print the results
cursor.execute("SELECT * FROM keywords LIMIT 10;")
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

Streaming output truncated to the last 5000 lines.
(2300, 6075, 'sport')
(2300, 6496, 'basketball')
(2300, 13005, 'doctor')
(2300, 176746, 'basketball team')
(2300, 177208, 'basketball game')
(2300, 177216, 'referee')
(2300, 177218, 'basketball court')
(2300, 177219, 'tweety bird')
(2300, 177224, 'speedy gonzales')
(2300, 177227, 'cartoon chicken')
(2300, 177228, 'sylvester the cat')
(2300, 177230, 'cartoon reality crossover')
(2300, 177231, 'basketball hoop')
(2300, 177232, 'cartoon skunk')
(8069, 9835, 'sexual fantasy')
(8069, 10158, 'alien planet')
(8069, 11239, 'distant future')
(8069, 165580, 'cult classic')
(8069, 191144, 'female mercenary')
(288173, 570, 'rape')
(288173, 2483, 'nudity')
(288173, 11734, 'police corruption')
(288173, 179571, 'tijuana')
(288173, 187056, 'woman director')
(27768, 10183, 'independent film')
(221917, 14985, 'ireland')
(221917, 194000, 'strained relationship')
(8072, 4565, 'dystopia')
(8072, 9887, 'surrealism')
(8072, 155517, 'lemmy caution')
(8072, 15

KeyboardInterrupt: 